In [1]:
import torch
import torch.nn as nn

from model_class import ResNet
from dataset_utils import get_data_loader
from train import out_chan_dict


def test(net, batch_size, num_classes, device):
    testloader = get_data_loader('test', batch_size, num_classes, shuffle=True)
    criterion = nn.CrossEntropyLoss()

    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for _, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test accuracy=%.4f" % (test_loss / (num_val_steps), val_acc))
    return val_acc  # my addition. Originally returns None


device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
save_name = "/home/zl310/cs585_project/vmoe/model_unencrypted/unencrypted_performance.txt"
with open(save_name, 'w') as f_ptr:
    f_ptr.write("num_classes,classification_accuracy\n")
    for num_classes in out_chan_dict.keys():
        net = ResNet(num_classes, out_chan=out_chan_dict[num_classes]).to(device)
        net.load_state_dict(torch.load(f"./checkpoints/resnet_{num_classes}.pt"))
        f_ptr.write("%d,%.6f\n" % (num_classes, test(net, batch_size=64, num_classes=num_classes, device=device)))

Test Loss=0.5199, Test accuracy=0.8400
Test Loss=0.2723, Test accuracy=0.9150
Test Loss=0.5005, Test accuracy=0.8600
Test Loss=0.9180, Test accuracy=0.7275
Test Loss=1.2552, Test accuracy=0.6344
Test Loss=1.5819, Test accuracy=0.5781
Test Loss=1.8517, Test accuracy=0.5308
Test Loss=2.4011, Test accuracy=0.4309
Test Loss=2.7367, Test accuracy=0.3786
Test Loss=3.1339, Test accuracy=0.3281
